<a href="https://colab.research.google.com/github/RoyEHamlin/Websockets/blob/main/websockets_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Cryptocurrency feed from Binaance using Websockets
## Tutorial at : https://www.youtube.com/watch?v=z2ePTq-KTzQ
## Resources :
### https://pypi.org/project/websocket-client/
### https://websocket-client.readthedocs.io/en/latest/
### https://binance-docs.github.io/apidocs/futures/en/
### https://developers.binance.com/docs/binance-api/spot-detail/web-socket-streams/#klinecandlestick-streams (example)
### https://coinderivs.ams3.cdn.digitaloceanspaces.com/recent_liquidations.json (Binance)
### https://fapi.binance.com/fapi/v1/depth?symbol=BTCUSDT&limit=1000 (orderbook)

In [102]:
pip install websocket-client

In [103]:
import websocket, json

In [104]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [105]:
cc = 'btcusd'
interval = '1m'

socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

In [112]:
import pandas as pd

r1 = {
    "unix_time" : 1.1,
    "symbol" : "BTCUSDT",
    "open" : 1.2,
    "close" : 1.3,
    "high" : 1.4,
    "low" : 1.5,
    "vol" : 1.6,
    "buy_vol" : 1.7,
    "num_trades" : 1,
    "candle_close" : True
}

# creates the df, then drops the dummy row to cast columns as correct types ONLY.
df = pd.DataFrame(r1, index=['a'])
df = df.drop('a') # d
old_time = int(datetime.fromtimestamp(time.time()).strftime("%S")) // 5 # old time stamp, as 1/12 of a minute
new_time = int(datetime.fromtimestamp(time.time()).strftime("%S")) // 5 # new time stamp, as 1/12 of a minute

In [107]:
# from numpy.ma.extras import count_masked

def on_message(ws, message):
    msg = json.loads(message)

    # if msg is next multiple of "5", save this row.  Otherwise ignore.
    global old_time
    global new_time
    print(old_time)
    # print(int(datetime.fromtimestamp(msg["E"]).strftime("%S")))
    # print(int(datetime.fromtimestamp(msg["E"]/1000).strftime("%S")))
    new_time = int(datetime.fromtimestamp(msg["E"]/1000).strftime("%S")) // 5
    print(new_time)
    # print(f"old:{old_time} and new:{new_time}")
    if old_time != new_time:
        old_time = new_time
    
        row = {
            "unix_time" : msg["E"]/1000,
            "symbol" : msg["k"]["s"],
            "open" : msg["k"]["o"],
            "close" : msg["k"]["c"],
            "high" : msg["k"]["h"],
            "low" : msg["k"]["l"],
            "vol" : msg["k"]["v"],
            "buy_vol" : msg["k"]["V"],
            "num_trades" : int(msg["k"]["n"]),
            "candle_close" : bool(msg["k"]["x"])
        }

        global df
        df = df.append(row, ignore_index=True)

        df.to_csv("btcprices.csv", index=False, encoding='utf-8-sig')





def on_close(ws):
    print("Connection Closed")

def on_open(ws):
    def run(*args):
        for i in range(3):
            time.sleep(1)
            ws.send("Hello %d" % i)
        time.sleep(1)
        ws.close()
        print("thread terminating...")
    _thread.start_new_thread(run, ())



ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close)

In [113]:
ws.run_forever()


0
1
1
1
1
2
2
2
2
3
3
3
3
3
3
4
4
4
4
5
5
5
5
6
6
6
6
6
6
7
7
7
7
8
8
8
8
9
9
9
9
9
9
10
10
10
10
11
11
11
11
0
0
0
0
0
0
1
1
1
1
2
2
2
2
3
3
3
3
4
4
4
4
5
5
5
5
6
6
6
6
6
6
7
7
7
7
8
8
8
8
9
9
9
9
10
10
10
10
11
11
11
11
11
11
0
0
0
0
0


False

In [120]:
 # https://towardsdatascience.com/google-colab-import-and-export-datasets-eccf801e2971
 df.to_csv("/btcprices.csv", index=False, encoding='utf-8-sig')
 df
 from google.colab import files
 df.to_csv('sample.csv')
 files.download('sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>